## Import Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager  
import re
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time


In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [50]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from datetime import datetime
import json
from pathlib import Path

import ast
import re
from datetime import datetime
import pandas as pd
from pathlib import Path
import json


## All required Functions

In [ ]:
# Initialize undetected Chrome driver
def init_driver(url):
    options = uc.ChromeOptions() 
    driver = uc.Chrome(options=options) 
    
    driver.get(url)
    return driver

# Close the driver
def close_driver(driver):
    driver.quit()

# Handle cookie consent and set filters
def handle_cookie_and_filters(driver):
    # Handle cookie consent
    try:
        wait = WebDriverWait(driver, 10)
        
        # Wait for the button to be clickable
        cookie_button = wait.until(
            EC.element_to_be_clickable((By.CLASS_NAME, "mde-consent-accept-btn"))
        )
        
        cookie_button.click()
        
        # Short pause for the overlay to fully disappear
        time.sleep(random.uniform(2, 4)) 
        
    except Exception as e:
        print(f'No cookie prompt found or could not interact with it: {e}')
        pass

# Set filters
def set_filters(driver):
    
    wait = WebDriverWait(driver, 10) 
    
    # # set location
    input_id = 'geolocation-autosuggest'
    # 1. Wait until the element is PRESENT in the DOM
    search_input = wait.until(
        EC.presence_of_element_located((By.ID, input_id))
    )
    
    # 2. Now that the element is found, we can interact with it
    search_input.send_keys('Kiel, Schleswig-Holstein')
    time.sleep(random.uniform(2, 3))
    search_input.send_keys(Keys.RETURN)
    

    # wait until button is in the DOM
    filter_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="qs-submit-button"]'))
    )

    # Explicitly click the button using JavaScript to bypass "not clickable" overlay issues
    driver.execute_script("arguments[0].click();", filter_button)

 # Scrape car listings   
def scrape_cars(driver):
    all_cars = []

    while True:
        try:
            time.sleep(random.uniform(3, 5))
            
            # Select all listing articles and divs with dispaly: contents
            # Exclude if there is a parent article with data-testid="srx-result-list-container"
            all_listings = driver.find_elements(
                By.XPATH,
                '//article[not(@data-testid="srx-result-list-container") \
                    and not(ancestor::article[@data-testid="srx-result-list-container"])] | \
                 //div[@style="display:contents"]/article[not(@data-testid="srx-result-list-container") \
                     and not(ancestor::article[@data-testid="srx-result-list-container"])]'
            )

            for car in all_listings:
                # print("Processing a car listing...")
                try:
                    title = car.find_element(By.XPATH, './/h2/span[1]').text.strip()
                    if title.lower() == "sponsored":
                        continue

                    subtitle = car.find_element(By.XPATH, './/h2/span[2]').text.strip()
                    price = car.find_element(By.CSS_SELECTOR, '[data-testid="price-label"]').text.strip()
                except:
                    continue

                try:
                    details = car.find_element(By.CSS_SELECTOR, '[data-testid^="listing-details-attributes"] div').text
                except:
                    details = None
                try:
                    seller = car.find_element(By.CSS_SELECTOR, '[data-testid="seller-info"]').text
                except:
                    seller = None
                try:
                    a_el = car.find_element(By.CSS_SELECTOR, 'a[href*="details.html"]')
                    href = a_el.get_attribute("href")
                except:
                    href = None

                car_data = {
                    "model": title,
                    "description": subtitle,
                    "price": re.sub(r'[^\d,]', '', price),
                    "specifications": details.split(' • ') if details else [],
                    "seller": seller,
                    "car_link": href
                }

                if not any(c['model'] == car_data['model'] and
                           c['description'] == car_data['description'] and
                           c['price'] == car_data['price'] for c in all_cars):
                    all_cars.append(car_data)

            try:
                continue_button = driver.find_element(By.CSS_SELECTOR, '[data-testid="pagination:next"]') # Next button
                if continue_button.is_enabled():
                    continue_button.click()
                    time.sleep(random.uniform(3, 5))
                else:
                    break
            except:
                break

        except Exception as e:
            print("Error while fetching listings:", e)
            break

    print(f"Total cars scraped: {len(all_cars)}")
    return all_cars

           

In [20]:
BASE_URL = 'https://www.mobile.de/?lang=en'

def main(url):
    driver = init_driver(url)
    try:
        handle_cookie_and_filters(driver)
        set_filters(driver)
        all_cars = scrape_cars(driver)
    finally:
        close_driver(driver)  # always close even if error
    return pd.DataFrame(all_cars)

cars = main(BASE_URL)
cars.head()


Total cars scraped: 985


,model,description,price,specifications,seller,car_link
0,Mercedes-Benz CLA 200 Shooting Brake,7G-DCT AMG Line+LED+NAVI,"18,999","[FR 09/2017, 92,215 km, 115 kW (156 hp), Petrol]",Kamux Auto GmbH\n4.3 stars\n(161)\n25746 Heide...,https://suchen.mobile.de/fahrzeuge/details.htm...
1,Mercedes-Benz E 200,CGI T 9G-tronic Avantgarde+LED+VIRTUAL+AHK,"27,750","[FR 01/2018, 71,293 km, 135 kW (184 hp), Petrol]",Kamux Auto GmbH\n4.9 stars\n(75)\n21682 Stade\...,https://suchen.mobile.de/fahrzeuge/details.htm...
2,Volkswagen Tiguan,2.0TSI 4Motion DSG Highline+LED+NAVI+HUD,"20,989","[FR 02/2017, 88,855 km, 132 kW (179 hp), Petrol]",Kamux Auto GmbH\n4.6 stars\n(611)\n22529 Hambu...,https://suchen.mobile.de/fahrzeuge/details.htm...
3,NEW,Mercedes-Benz C 220,"23,450","[FR 11/2018, 103,386 km, 143 kW (194 hp), Diesel]",Kamux Auto GmbH\n4.6 stars\n(611)\n22529 Hambu...,https://suchen.mobile.de/fahrzeuge/details.htm...
4,Mercedes-Benz GLC 43 AMG,4Matic PANO/ACC/AHK,"35,900","[Accident-free, FR 09/2018, 119,390 km, 270 kW...",H&S Automobile Hamburg GMBH\n4.6 stars\n(102)\...,https://suchen.mobile.de/fahrzeuge/details.htm...


In [ ]:
# Save to CSV
cars.to_csv('cars.csv', index=False, encoding='utf-8-sig')

print("CSV saved successfully.")

CSV saved successfully.


In [ ]:
# Display DataFrame info
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model           985 non-null    object
 1   description     985 non-null    object
 2   price           985 non-null    object
 3   specifications  985 non-null    object
 4   seller          985 non-null    object
 5   car_link        985 non-null    object
dtypes: object(6)
memory usage: 46.3+ KB


In [ ]:
# Display summary statistics
cars.describe()

,model,description,price,specifications,seller,car_link
count,985,985,985,985,985,985
unique,188,902,529,977,339,985
top,NEW,Volkswagen Golf,"25,990","[Accident-free, FR 08/2017, 64,271 km, 155 kW ...",Autohero GmbH - Standort Hamburg\n4.5 stars\n(...,https://suchen.mobile.de/fahrzeuge/details.htm...
freq,153,15,16,3,74,1


In [ ]:
# Display value counts for 'model' column
cars['model'].value_counts()

model
NEW                        153
Volkswagen Golf             65
Volkswagen Tiguan           37
Mercedes-Benz C 180         30
Mercedes-Benz A 180         23
                          ... 
BMW 640                      1
Peugeot 208                  1
Skoda Kodiaq                 1
Mercedes-Benz CL 500         1
Volkswagen T6 Caravelle      1
Name: count, Length: 188, dtype: int64

In [ ]:
# Create a copy for preprocessing
cars_preprocess = cars.copy()

There are many "NEW" in model column and i observe that the description column has the actual model name. so i swapped it and update the description as 'Not provided by the seller' 

In [ ]:
rows_to_update = cars_preprocess['model'] == 'NEW'

cars_preprocess.loc[rows_to_update, 'model'] = cars_preprocess['description']
cars_preprocess.loc[rows_to_update, 'description'] = 'Not provided by seller.'

In [31]:
cars_preprocess['model'].value_counts()

model
Volkswagen Golf            80
Volkswagen Tiguan          46
Mercedes-Benz C 180        34
Mercedes-Benz E 200        26
Mercedes-Benz A 200        25
                           ..
BMW 640                     1
Peugeot 208                 1
Skoda Kodiaq                1
Mercedes-Benz CL 500        1
Volkswagen T6 Caravelle     1
Name: count, Length: 202, dtype: int64

In [ ]:
# Display DataFrame info after preprocessing
cars_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model           985 non-null    object
 1   description     985 non-null    object
 2   price           985 non-null    object
 3   specifications  985 non-null    object
 4   seller          985 non-null    object
 5   car_link        985 non-null    object
dtypes: object(6)
memory usage: 46.3+ KB


Converts string-based structured data (lists, dicts) into Python objects, handling missing values (to []), correcting smart quote errors, and serving as a fail-safe by returning [] on any parsing error.

In [38]:
def safe_literal_eval(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        return ast.literal_eval(x)
    except Exception:
        # Try a cleaned version (replace smart quotes)
        try:
            cleaned = x.replace("“", '"').replace("”", '"').replace("’", "'")
            return ast.literal_eval(cleaned)
        except Exception:
            return []

Parse price by removing all "," and convert into float

In [39]:
def parse_price(p):
    if pd.isna(p):
        return None
    s = str(p).strip()
    # remove commas
    s = s.replace(",", "")

    if s == "":
        return None
    try:
        val = float(s)
        return int(round(val))
    except Exception:
        return None


Let's spilt the specifications coulmn into individual columns

In [ ]:
def parse_specifications(spec_list):
    out = {
        "first_registration": None,
        "mileage_km": None,
        "power_kw": None,
        "power_hp": None,
        "fuel_type": None,
        "accident_free": False,
    }
    if not spec_list:
        return out

    for item in spec_list:
        if not isinstance(item, str):
            continue
        s = item.strip()

        # 1) First registration: look for 'MM/YYYY'
        m = re.search(r"(\d{2})[\/\.](\d{4})", s)
        if m and out["first_registration"] is None:
            month = int(m.group(1))
            year = int(m.group(2))
            try:
                out["first_registration"] = f"{year:04d}-{month:02d}"
            except Exception:
                out["first_registration"] = None
            continue

        # 2) Mileage: look for '... km'
        if "km" in s.lower() and out["mileage_km"] is None:
            # remove non-digits
            num = re.sub(r"[^\d]", "", s)
            if num != "":
                out["mileage_km"] = int(num)
            continue

        # 3) Power: '115 kW (156 hp)'
        if "kW" in s or "kw" in s or "hp" in s:
            mkw = re.search(r"(\d+)\s*[kK][Ww]", s)
            if mkw:
                out["power_kw"] = int(mkw.group(1))
            mhp = re.search(r"(\d+)\s*[hH][pP]", s) 
            if mhp:
                out["power_hp"] = int(mhp.group(1))
            continue

        # 4) Fuel type detection (simple)
        if any(k in s.lower() for k in ["petrol", "diesel", "electric", "hybrid"]):
            out["fuel_type"] = s.split(",")[0].strip()
            continue

        # 5) Accident-free detection
        if re.search(r'accident[-\s]?free', s, flags=re.I):
            out["accident_free"] = True
        else:
            out["accident_free"] = False
    return out


Fununtion split the sellers column

In [41]:
def parse_seller(raw_text):
    # return empty if data is missing
    if not isinstance(raw_text, str) or not raw_text.strip():
        return None
        
    # Split the text by new lines and remove empty lines
    lines = [line.strip() for line in raw_text.split('\n') if line.strip()]
    
    data = {
        "seller_name": None,
        "seller_rating": None,
        "seller_reviews_count": None,
        "seller_pincode": None,
        "seller_city": None
    }

    # 1. Extract Seller Name (Usually the first line)
    if len(lines) > 0:
        data["seller_name"] = lines[0]

    # 2. Extract Rating (Look for line containing "stars")
    # We use regex to find a decimal or integer at the start of the line followed by 'stars'
    rating_match = next((line for line in lines if "stars" in line), None)
    if rating_match:
        # Extract just the number (e.g., "4.9" from "4.9 stars")
        data["seller_rating"] = rating_match.replace(" stars", "").strip()

    # 3. Extract No. of Ratings (Look for line with parentheses, e.g., "(70)")
    count_match = next((line for line in lines if line.startswith("(") and line.endswith(")")), None)
    if count_match:
        # Remove parentheses
        data["seller_reviews_count"] = count_match.replace("(", "").replace(")", "")

    # 4. Extract Pincode and City (Look for the German Zip Code pattern: 5 digits)
    # Regex: Start with 5 digits (\d{5}), followed by whitespace (\s+), then the rest is the city (.*)
    address_line = next((line for line in lines if re.match(r'^\d{5}\s', line)), None)
    if address_line:
        match = re.search(r'^(\d{5})\s+(.*)', address_line)
        if match:
            data["seller_pincode"] = match.group(1)
            data["seller_city"] = match.group(2)

    return data

Normalise the model

In [42]:
def normalize_model(model_raw):
    if pd.isna(model_raw):
        return None, None
    s = str(model_raw).strip()
    if s.upper() in {"NEW", "—", "-", ""}:
        return None, None
    # collapse repeated whitespace/newlines into single spaces
    s = re.sub(r"\s+", " ", s)
    tokens = s.split(" ")
    brand = tokens[0] if tokens else None
    return brand, s

In [43]:
# Trim whitespace in main text columns (defensive)
for c in ["model", "description", "price", "specifications", "seller"]:
    if c in cars_preprocess.columns:
        cars_preprocess[c] = cars_preprocess[c].astype(str).str.strip()

In [ ]:
# Parse specifications: convert list-like string -> list, then extract fields

cars_preprocess["spec_list"] = cars_preprocess["specifications"].apply(safe_literal_eval)
specs_parsed = cars_preprocess["spec_list"].apply(parse_specifications).apply(pd.Series)
cars_preprocess = pd.concat([cars_preprocess, specs_parsed], axis=1)


# Parse price into numeric eur
cars_preprocess["price_eur"] = cars_preprocess["price"].apply(parse_price)

# Parse seller column into structured fields
seller_parsed = cars_preprocess["seller"].apply(parse_seller).apply(pd.Series)
cars_preprocess = pd.concat([cars_preprocess, seller_parsed], axis=1)

# Normalize model and extract brand
brand_model = cars_preprocess["model"].apply(lambda x: pd.Series(normalize_model(x), index=["brand", "model_clean"]))
cars_preprocess = pd.concat([cars_preprocess, brand_model], axis=1)

In [46]:
cars_preprocess.head()


,model,description,price,specifications,seller,car_link,spec_list,first_registration,mileage_km,power_kw,...,fuel_type,accident_free,price_eur,seller_name,seller_rating,seller_reviews_count,seller_pincode,seller_city,brand,model_clean
0,Mercedes-Benz CLA 200 Shooting Brake,7G-DCT AMG Line+LED+NAVI,"18,999","['FR 09/2017', '92,215 km', '115 kW (156 hp)',...",Kamux Auto GmbH\n4.3 stars\n(161)\n25746 Heide...,https://suchen.mobile.de/fahrzeuge/details.htm...,"[FR 09/2017, 92,215 km, 115 kW (156 hp), Petrol]",2017-09,92215,115.0,...,Petrol,False,18999,Kamux Auto GmbH,4.3,161,25746,Heide,Mercedes-Benz,Mercedes-Benz CLA 200 Shooting Brake
1,Mercedes-Benz E 200,CGI T 9G-tronic Avantgarde+LED+VIRTUAL+AHK,"27,750","['FR 01/2018', '71,293 km', '135 kW (184 hp)',...",Kamux Auto GmbH\n4.9 stars\n(75)\n21682 Stade\...,https://suchen.mobile.de/fahrzeuge/details.htm...,"[FR 01/2018, 71,293 km, 135 kW (184 hp), Petrol]",2018-01,71293,135.0,...,Petrol,False,27750,Kamux Auto GmbH,4.9,75,21682,Stade,Mercedes-Benz,Mercedes-Benz E 200
2,Volkswagen Tiguan,2.0TSI 4Motion DSG Highline+LED+NAVI+HUD,"20,989","['FR 02/2017', '88,855 km', '132 kW (179 hp)',...",Kamux Auto GmbH\n4.6 stars\n(611)\n22529 Hambu...,https://suchen.mobile.de/fahrzeuge/details.htm...,"[FR 02/2017, 88,855 km, 132 kW (179 hp), Petrol]",2017-02,88855,132.0,...,Petrol,False,20989,Kamux Auto GmbH,4.6,611,22529,Hamburg,Volkswagen,Volkswagen Tiguan
3,Mercedes-Benz C 220,Not provided by seller.,"23,450","['FR 11/2018', '103,386 km', '143 kW (194 hp)'...",Kamux Auto GmbH\n4.6 stars\n(611)\n22529 Hambu...,https://suchen.mobile.de/fahrzeuge/details.htm...,"[FR 11/2018, 103,386 km, 143 kW (194 hp), Diesel]",2018-11,103386,143.0,...,Diesel,False,23450,Kamux Auto GmbH,4.6,611,22529,Hamburg,Mercedes-Benz,Mercedes-Benz C 220
4,Mercedes-Benz GLC 43 AMG,4Matic PANO/ACC/AHK,"35,900","['Accident-free', 'FR 09/2018', '119,390 km', ...",H&S Automobile Hamburg GMBH\n4.6 stars\n(102)\...,https://suchen.mobile.de/fahrzeuge/details.htm...,"[Accident-free, FR 09/2018, 119,390 km, 270 kW...",2018-09,119390,270.0,...,Petrol,True,35900,H&S Automobile Hamburg GMBH,4.6,102,23795,Bad Segeberg,Mercedes-Benz,Mercedes-Benz GLC 43 AMG


Derive some important features

In [ ]:
# 1) Car age in full years 
def calc_age(reg):
    # Handle missing / non-scalar values safely
    try:
        if reg is None:
            return None
        # if reg is a Series/list/ndarray, bail out (not a single date string)
        if isinstance(reg, (list, tuple, pd.Series, pd.Index, pd.DataFrame)):
            return None
        # guard for NaN-like values
        if pd.isna(reg):
            return None
        # parse
        dt = datetime.strptime(str(reg), "%Y-%m")
        today = datetime.today()
        years = today.year - dt.year - ((today.month, today.day) < (dt.month, dt.day))
        return years
    except Exception:
        return None

cars_preprocess["car_age_years"] = cars_preprocess["first_registration"].apply(calc_age)


# 2) Mileage category for quick bucketing
def mileage_cat(m):
    if pd.isna(m) or m is None:
        return None
    try:
        m = int(m)
        if m < 50000:
            return "<50k"
        if m <= 100000:
            return "50k-100k"
        return ">100k"
    except Exception:
        return None

cars_preprocess["mileage_category"] = cars_preprocess["mileage_km"].apply(mileage_cat)

In [52]:
cars_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   model                 985 non-null    object 
 1   description           985 non-null    object 
 2   price                 985 non-null    object 
 3   specifications        985 non-null    object 
 4   seller                985 non-null    object 
 5   car_link              985 non-null    object 
 6   spec_list             985 non-null    object 
 7   first_registration    985 non-null    object 
 8   mileage_km            985 non-null    int64  
 9   power_kw              984 non-null    float64
 10  power_hp              984 non-null    float64
 11  fuel_type             985 non-null    object 
 12  accident_free         985 non-null    bool   
 13  price_eur             985 non-null    int64  
 14  seller_name           985 non-null    object 
 15  seller_rating         9

Build a single text field for RAG ingestion (concise summary of important attributes + description)

In [ ]:
def build_rag_text(row):
    parts = []
    if row.get("brand"):
        parts.append(str(row["brand"]))
    if row.get("model_clean"):
        parts.append(str(row["model_clean"]))
    if row.get("first_registration"):
        parts.append(f"First registration: {row['first_registration']}")
    accident_status = row.get("accident_free")
    if accident_status is True:
        parts.append(f"Accident-free: Yes")
    elif accident_status is False:
        parts.append(f"Accident-free: No")
    if row.get("mileage_km"):
        try:
            parts.append(f"Mileage: {int(row['mileage_km']):,} km")
        except Exception:
            parts.append(f"Mileage: {row['mileage_km']} km")
    if row.get("fuel_type"):
        parts.append(f"Fuel: {row['fuel_type']}")
    if row.get("power_kw"):
        parts.append(f"Power: {row['power_kw']} kW")
    if row.get("price_eur"):
        parts.append(f"Price: €{int(row['price_eur']):,}")
    if row.get("seller_name"):
        seller = row["seller_name"]
        if row.get("seller_rating"):
            seller += f" ({row['seller_rating']}⭐)"
        parts.append(f"Seller: {seller}")
    # Original description 
    if row.get("description") and str(row["description"]).strip().lower() != "nan":
        parts.append("Description: " + str(row["description"]).strip())
    return " | ".join(parts)

cars_preprocess["rag_text"] = cars_preprocess.apply(build_rag_text, axis=1)

In [ ]:
# Display first 5 rows with full column width
pd.set_option('display.max_colwidth', None)  # truncate long strings
cars_preprocess.head(5)

,model,description,price,specifications,seller,car_link,spec_list,first_registration,mileage_km,power_kw,...,seller_name,seller_rating,seller_reviews_count,seller_pincode,seller_city,brand,model_clean,car_age_years,mileage_category,rag_text
0,Mercedes-Benz CLA 200 Shooting Brake,7G-DCT AMG Line+LED+NAVI,"18,999","['FR 09/2017', '92,215 km', '115 kW (156 hp)', 'Petrol']",Kamux Auto GmbH\n4.3 stars\n(161)\n25746 Heide\nContact\nPark,https://suchen.mobile.de/fahrzeuge/details.html?id=442028837&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,"[FR 09/2017, 92,215 km, 115 kW (156 hp), Petrol]",2017-09,92215,115.0,...,Kamux Auto GmbH,4.3,161,25746,Heide,Mercedes-Benz,Mercedes-Benz CLA 200 Shooting Brake,8,50k-100k,"Mercedes-Benz | Mercedes-Benz CLA 200 Shooting Brake | First registration: 2017-09 | Accident-free: No | Mileage: 92,215 km | Fuel: Petrol | Power: 115.0 kW | Price: €18,999 | Seller: Kamux Auto GmbH (4.3⭐) | Description: 7G-DCT AMG Line+LED+NAVI"
1,Mercedes-Benz E 200,CGI T 9G-tronic Avantgarde+LED+VIRTUAL+AHK,"27,750","['FR 01/2018', '71,293 km', '135 kW (184 hp)', 'Petrol']",Kamux Auto GmbH\n4.9 stars\n(75)\n21682 Stade\nContact\nPark,https://suchen.mobile.de/fahrzeuge/details.html?id=441361076&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,"[FR 01/2018, 71,293 km, 135 kW (184 hp), Petrol]",2018-01,71293,135.0,...,Kamux Auto GmbH,4.9,75,21682,Stade,Mercedes-Benz,Mercedes-Benz E 200,7,50k-100k,"Mercedes-Benz | Mercedes-Benz E 200 | First registration: 2018-01 | Accident-free: No | Mileage: 71,293 km | Fuel: Petrol | Power: 135.0 kW | Price: €27,750 | Seller: Kamux Auto GmbH (4.9⭐) | Description: CGI T 9G-tronic Avantgarde+LED+VIRTUAL+AHK"
2,Volkswagen Tiguan,2.0TSI 4Motion DSG Highline+LED+NAVI+HUD,"20,989","['FR 02/2017', '88,855 km', '132 kW (179 hp)', 'Petrol']",Kamux Auto GmbH\n4.6 stars\n(611)\n22529 Hamburg\nContact\nPark,https://suchen.mobile.de/fahrzeuge/details.html?id=441664842&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,"[FR 02/2017, 88,855 km, 132 kW (179 hp), Petrol]",2017-02,88855,132.0,...,Kamux Auto GmbH,4.6,611,22529,Hamburg,Volkswagen,Volkswagen Tiguan,8,50k-100k,"Volkswagen | Volkswagen Tiguan | First registration: 2017-02 | Accident-free: No | Mileage: 88,855 km | Fuel: Petrol | Power: 132.0 kW | Price: €20,989 | Seller: Kamux Auto GmbH (4.6⭐) | Description: 2.0TSI 4Motion DSG Highline+LED+NAVI+HUD"
3,Mercedes-Benz C 220,Not provided by seller.,"23,450","['FR 11/2018', '103,386 km', '143 kW (194 hp)', 'Diesel']",Kamux Auto GmbH\n4.6 stars\n(611)\n22529 Hamburg\nContact\nPark,https://suchen.mobile.de/fahrzeuge/details.html?id=440147447&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,"[FR 11/2018, 103,386 km, 143 kW (194 hp), Diesel]",2018-11,103386,143.0,...,Kamux Auto GmbH,4.6,611,22529,Hamburg,Mercedes-Benz,Mercedes-Benz C 220,7,>100k,"Mercedes-Benz | Mercedes-Benz C 220 | First registration: 2018-11 | Accident-free: No | Mileage: 103,386 km | Fuel: Diesel | Power: 143.0 kW | Price: €23,450 | Seller: Kamux Auto GmbH (4.6⭐) | Description: Not provided by seller."
4,Mercedes-Benz GLC 43 AMG,4Matic PANO/ACC/AHK,"35,900","['Accident-free', 'FR 09/2018', '119,390 km', '270 kW (367 hp)', 'Petrol']",H&S Automobile Hamburg GMBH\n4.6 stars\n(102)\n23795 Bad Segeberg\nContact\nPark,https://suchen.mobile.de/fahrzeuge/details.ht

In [ ]:
# Display the RAG text of the first car for verification
cars_preprocess['rag_text'].iloc[0]

'Mercedes-Benz | Mercedes-Benz CLA 200 Shooting Brake | First registration: 2017-09 | Accident-free: No | Mileage: 92,215 km | Fuel: Petrol | Power: 115.0 kW | Price: €18,999 | Seller: Kamux Auto GmbH (4.3⭐) | Description: 7G-DCT AMG Line+LED+NAVI'

check for duplicates

In [ ]:
# Remove duplicates based on key fields
cars_preprocess = cars_preprocess.drop_duplicates(subset=["brand", "model_clean", "mileage_km", "price_eur", "seller_name"])

In [ ]:
# Prepare final output table (select & order columns)
cols_order = [
    "brand", "model_clean", "car_link", "description", "price", "price_eur", "first_registration", "accident_free", "car_age_years",
    "mileage_km", "mileage_category", "power_kw", "power_hp", "fuel_type", "transmission",
    "seller_name", "seller_rating", "seller_reviews_count", "seller_location",
    "price_per_km", "rag_text", "specifications", "spec_list"
]
# keep only existing columns (defensive)
cols_order = [c for c in cols_order if c in cars_preprocess.columns]
cars_RAG = cars_preprocess[cols_order].copy()

In [81]:
cars_RAG.head()

,brand,model_clean,car_link,description,price,price_eur,first_registration,accident_free,car_age_years,mileage_km,mileage_category,power_kw,power_hp,fuel_type,seller_name,seller_rating,seller_reviews_count,rag_text,specifications,spec_list
0,Mercedes-Benz,Mercedes-Benz CLA 200 Shooting Brake,https://suchen.mobile.de/fahrzeuge/details.html?id=442028837&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,7G-DCT AMG Line+LED+NAVI,"18,999",18999,2017-09,False,8,92215,50k-100k,115.0,156.0,Petrol,Kamux Auto GmbH,4.3,161,"Mercedes-Benz | Mercedes-Benz CLA 200 Shooting Brake | First registration: 2017-09 | Accident-free: No | Mileage: 92,215 km | Fuel: Petrol | Power: 115.0 kW | Price: €18,999 | Seller: Kamux Auto GmbH (4.3⭐) | Description: 7G-DCT AMG Line+LED+NAVI","['FR 09/2017', '92,215 km', '115 kW (156 hp)', 'Petrol']","[FR 09/2017, 92,215 km, 115 kW (156 hp), Petrol]"
1,Mercedes-Benz,Mercedes-Benz E 200,https://suchen.mobile.de/fahrzeuge/details.html?id=441361076&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,CGI T 9G-tronic Avantgarde+LED+VIRTUAL+AHK,"27,750",27750,2018-01,False,7,71293,50k-100k,135.0,184.0,Petrol,Kamux Auto GmbH,4.9,75,"Mercedes-Benz | Mercedes-Benz E 200 | First registration: 2018-01 | Accident-free: No | Mileage: 71,293 km | Fuel: Petrol | Power: 135.0 kW | Price: €27,750 | Seller: Kamux Auto GmbH (4.9⭐) | Description: CGI T 9G-tronic Avantgarde+LED+VIRTUAL+AHK","['FR 01/2018', '71,293 km', '135 kW (184 hp)', 'Petrol']","[FR 01/2018, 71,293 km, 135 kW (184 hp), Petrol]"
2,Volkswagen,Volkswagen Tiguan,https://suchen.mobile.de/fahrzeuge/details.html?id=441664842&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,2.0TSI 4Motion DSG Highline+LED+NAVI+HUD,"20,989",20989,2017-02,False,8,88855,50k-100k,132.0,179.0,Petrol,Kamux Auto GmbH,4.6,611,"Volkswagen | Volkswagen Tiguan | First registration: 2017-02 | Accident-free: No | Mileage: 88,855 km | Fuel: Petrol | Power: 132.0 kW | Price: €20,989 | Seller: Kamux Auto GmbH (4.6⭐) | Description: 2.0TSI 4Motion DSG Highline+LED+NAVI+HUD","['FR 02/2017', '88,855 km', '132 kW (179 hp)', 'Petrol']","[FR 02/2017, 88,855 km, 132 kW (179 hp), Petrol]"
3,Mercedes-Benz,Mercedes-Benz C 220,https://suchen.mobile.de/fahrzeuge/details.html?id=440147447&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,Not provided by seller.,"23,450",23450,2018-11,False,7,103386,>100k,143.0,194.0,Diesel,Kamux Auto GmbH,4.6,611,"Mercedes-Benz | Mercedes-Benz C 220 | First registration: 2018-11 | Accident-free: No | Mileage: 103,386 km | Fuel: Diesel | Power: 143.0 kW | Price: €23,450 | Seller: Kamux Auto GmbH (4.6⭐) | Description: Not provided by seller.","['FR 11/2018', '103,386 km', '143 kW (194 hp)', 'Diesel']","[FR 11/2018, 103,386 km, 143 kW (194 hp), Diesel]"
4,Mercedes-Benz,Mercedes-Benz GLC 43 AMG,https://suchen.mobile.de/fahrzeuge/details.html?id=441104549&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car,4Matic PANO/ACC/AHK,"35,900",35900,2018-09,True,7,119390,>100k,270.0,367.0,Petrol,H&S Automobile Hamburg GMBH,4.6,102,"Mercedes-Benz | Mercedes-Benz GLC 43 AMG | First registration: 2018-09 | Accident-free: Yes | Mileage: 119,390 km | Fuel: Petrol | P

In [82]:
cars_RAG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   brand                 985 non-null    object 
 1   model_clean           985 non-null    object 
 2   car_link              985 non-null    object 
 3   description           985 non-null    object 
 4   price                 985 non-null    object 
 5   price_eur             985 non-null    int64  
 6   first_registration    985 non-null    object 
 7   accident_free         985 non-null    bool   
 8   car_age_years         985 non-null    int64  
 9   mileage_km            985 non-null    int64  
 10  mileage_category      985 non-null    object 
 11  power_kw              984 non-null    float64
 12  power_hp              984 non-null    float64
 13  fuel_type             985 non-null    object 
 14  seller_name           985 non-null    object 
 15  seller_rating         9

let's save the outputs as CSV and JSOL for building RAG

In [84]:
cars_RAG_CSV = Path("D:\\FH Pdfs\\2nd semester\\Social Media Analytics\\Submissions\\capstone_sample\\cars_RAG.csv")
cars_RAG_JSONL = Path("D:\\FH Pdfs\\2nd semester\\Social Media Analytics\\Submissions\\capstone_sample\\cars_RAG.jsonl")

In [85]:
cars_RAG.to_csv(cars_RAG_CSV, index=False)

In [87]:
# JSONL for RAG ingestion: id, text, meta
with open(cars_RAG_JSONL, "w", encoding="utf-8") as f:
    for idx, r in cars_RAG.reset_index(drop=True).iterrows():
        doc = {
            "id": f"car_{idx}",
            "text": r.get("rag_text", ""),
            "meta": {
                "brand": r.get("brand"),
                "model": r.get("model_clean"),
                "price_eur": int(r["price_eur"]) if not pd.isna(r.get("price_eur")) else None,
                "first_registration": r.get("first_registration"),
                "mileage_km": int(r["mileage_km"]) if not pd.isna(r.get("mileage_km")) else None,
                "car_link": r.get("car_link"),
                "accident_free": r.get("accident_free"),
                "car_age_years": int(r["car_age_years"]) if not pd.isna(r.get("car_age_years")) else None,
                "mileage_category": r.get("mileage_category"),
                "power_kw": float(r["power_kw"]) if not pd.isna(r.get("power_kw")) else None,
                "fuel_type": r.get("fuel_type"),
                "seller_name": r.get("seller_name"),
                "seller_rating": float(r["seller_rating"]) if not pd.isna(r.get("seller_rating")) else None,
                "seller_reviews_count": int(r["seller_reviews_count"]) if not pd.isna(r.get("seller_reviews_count")) else None
            }
        }
        f.write(json.dumps(doc, ensure_ascii=False) + "\n")